In [1]:
import json
import os
import random
import numpy as np
from tqdm import tqdm

import torch
from tqdm import tqdm
import random
import seaborn as sns
from pylab import rcParams

from SkeletonData.data import *

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

In [2]:
npz_files = "../data/skel_out/hmdb51"

### Add Skeleton of selected videos

In [3]:
#action_npz_file = "../data/skeleton_k10_v7_ movements.npz"
#action_arr = np.load(action_npz_file)

In [4]:
#for cls,arr,vid_n in zip(action_arr["arr_0"],action_arr["arr_1"],action_arr["arr_2"]):
#    os.makedirs(os.path.join(npz_files,f"{cls}"),exist_ok=True)
#    np.savez(os.path.join(npz_files,f"{cls}/{vid_n}.npz"),coords)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
mapping_l = [
        [15, 13], [13, 11], [11, 5],
        [12, 14], [14, 16], [12, 6],
        [3, 1],[1, 2],[1, 0],[0, 2],[2,4],
        [9, 7], [7,5], [5, 6],
        [6, 8], [8, 10],
        ]
#mapping_l = []

## Sequence Data

In [7]:
root_dir = "D:\\FYP\\HAR-ZSL-XAI\\AETraining"
main_dir = "D:\\FYP\\HAR-ZSL-XAI"
data_dir = os.path.join(main_dir,"data","ntu120_3danno.pkl")
refined_data = os.path.join(main_dir,"data","sequence_data","random_50f")
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 1-train_ratio - val_ratio
batch_size = 32

In [8]:
class_names = list(range(120))

In [9]:
file_list = np.load("E:\\FYP_Data\\NTU120\\file_list_above_60.npz")["files"]

In [10]:
len(file_list)

54836

In [11]:
from SkeletonData.array_segment import *
from SkeletonData.visualize import *

In [12]:
with open("E:\\FYP_Data\\NTU120\\shapes_keys.json","r") as f0:
    id2shapes = json.load(f0)

file_keys = np.load("E:\\FYP_Data\\NTU120\\file_list_above_60.npz")["files"].tolist()

In [13]:
pkl_reader = SkeletonPickleReader(file_path="E:\\FYP_Data\\NTU120\\ntu120_hrnet.pkl")
#pkl_reader = SkeletonPickleReader(file_path="E:\\FYP_Data\\NTU120\\ntu120_hrnet.pkl")
splits, anno = pkl_reader.generate()

In [14]:
files = FileDictIterator(anno)

### Visualize

In [15]:
from functools import partial
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [16]:
def visualize_after_normalize(__f):
    kp = __f["keypoint"][0]
    kp[:,:,0] = kp[:,:,0]/__f["original_shape"][1]
    kp[:,:,1] = kp[:,:,1]/__f["original_shape"][0]

    gen_video(kp,f"temp_files/{__f['frame_dir']}.mp4",__f["original_shape"][0],__f["original_shape"][1],mapping_list=mapping_l)

    return f"temp_files/{__f['frame_dir']}.mp4"

In [17]:
os.makedirs("temp_files",exist_ok=True)

#with ThreadPoolExecutor() as executor:
#    file_loc = list(tqdm(executor.map(visualize_after_normalize,files), total=len(files),desc="Saved Videos:"))

### Break Into Segments

In [18]:
files = iter(files)

In [19]:
os.makedirs(refined_data,exist_ok=True)
#for each_file in tqdm(files,desc="Files Used",total=len(files)):
with ThreadPoolExecutor() as executor:
    file_loc = list(tqdm(executor.map(partial(split_array_main,id2shapes,refined_data),files), total=len(files),desc="Processed Files:"))

Processed Files:: 100%|██████████| 113945/113945 [01:12<00:00, 1564.74it/s]
